In [1]:
!pip install transformers datasets sentencepiece accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from collections import defaultdict
import pandas as pd

In [3]:
def preprocess_squad_dataset(max_qa_per_context=5):
    dataset = load_dataset("squad", split="train")

    context_dict = defaultdict(list)

    for item in dataset:
        context = item["context"].strip()
        question = item["question"].strip()
        answer = item["answers"]["text"][0].strip() if item["answers"]["text"] else ""
        context_dict[context].append((question, answer))

    inputs, targets = [], []

    for context, qa_list in context_dict.items():
        if len(qa_list) < 2:
            continue
        qa_list = qa_list[:max_qa_per_context]

        input_text = f"generate questions and answers: {context}"
        target_text = "\n".join([f"Q{i+1}: {q}\nA{i+1}: {a}" for i, (q, a) in enumerate(qa_list)])
        inputs.append(input_text)
        targets.append(target_text)

    return inputs, targets

In [4]:
def preprocess_sciq_dataset(max_qa_per_context=3):
    dataset = load_dataset("sciq", split="train")

    context_dict = defaultdict(list)

    for item in dataset:
        context = item["support"].strip()
        question = item["question"].strip()
        answer = item["correct_answer"].strip()
        context_dict[context].append((question, answer))

    inputs, targets = [], []

    for context, qa_list in context_dict.items():
        if len(qa_list) < 2:
            continue
        qa_list = qa_list[:max_qa_per_context]

        input_text = f"generate questions and answers: {context}"
        target_text = "\n".join([f"Q{i+1}: {q}\nA{i+1}: {a}" for i, (q, a) in enumerate(qa_list)])
        inputs.append(input_text)
        targets.append(target_text)

    return inputs, targets

In [5]:
squad_inputs, squad_targets = preprocess_squad_dataset()
sciq_inputs, sciq_targets = preprocess_sciq_dataset()

all_inputs = squad_inputs + sciq_inputs
all_targets = squad_targets + sciq_targets

df = pd.DataFrame({'input': all_inputs, 'target': all_targets})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/339k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
df.to_csv("./qa_dataset.csv", index=False)

In [7]:
raw_dataset = load_dataset('csv', data_files='qa_dataset.csv')['train']

# Split: 90% train, 10% validation
split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset['train']
valid_dataset = split_dataset['test']

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [9]:
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)

max_input_length = 512
max_target_length = 256

# Fungsi tokenisasi
def preprocess_function(example):
    model_input = tokenizer(
        example['input'],
        max_length=max_input_length,
        padding="max_length",
        truncation=True,
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example['target'],
            max_length=max_target_length,
            padding="max_length",
            truncation=True,
        )
    model_input["labels"] = labels["input_ids"]
    return model_input

# tokenized_dataset = dataset.map(preprocess_function, batched=True)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir= "./t5-mqg-base",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.map(preprocess_function, batched=True),
    eval_dataset=valid_dataset.map(preprocess_function, batched=True),
    tokenizer=tokenizer,
)

Map:   0%|          | 0/16726 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1859 [00:00<?, ? examples/s]

<ipython-input-11-f247c48cda5b>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: liu-hengky-laurencio (liu-hengky-laurencio-universitas-tarumanagara) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.486300,0.433964
2,0.532100,0.493659
3,0.535900,0.479024


TrainOutput(global_step=12546, training_loss=0.5343335326657082, metrics={'train_runtime': 6010.4309, 'train_samples_per_second': 8.348, 'train_steps_per_second': 2.087, 'total_flos': 3.055628903251968e+16, 'train_loss': 0.5343335326657082, 'epoch': 3.0})

In [13]:
trainer.save_model("t5-mqg-base-model")
tokenizer.save_pretrained("t5-mqg-base-model")

('t5-mqg-base-model/tokenizer_config.json',
 't5-mqg-base-model/special_tokens_map.json',
 't5-mqg-base-model/spiece.model',
 't5-mqg-base-model/added_tokens.json')

In [15]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="./t5-mqg-base-model", tokenizer="./t5-mqg-base-model")

context = "generate questions and answers: The lungs are the main organs in the human respiratory system. When we inhale, oxygen from the air enters through the nose, passes through the throat, and reaches the lungs. In the lungs, oxygen is exchanged for carbon dioxide in the alveoli, and then carbon dioxide is released when we exhale. This process is called external respiration and is essential for human survival."
output = pipe(context, max_length=256, clean_up_tokenization_spaces=True)[0]['generated_text']

print(output)

Device set to use cuda:0


Q1: What are the main organs in the human respiratory system? A1: The lungs Q2: What is the process called when oxygen enters through the nose? A2: external respiration Q3: What is the process called when oxygen is exchanged for carbon dioxide? A3: carbon dioxide Q4: What is the process called when oxygen is exchanged for carbon dioxide? A4: alveoli Q5: What is the process called when carbon dioxide is released? A5: exhale


In [16]:
!zip -r t5-mqg-base-model.zip t5-mqg-base-model

  adding: t5-mqg-base-model/ (stored 0%)
  adding: t5-mqg-base-model/tokenizer_config.json (deflated 94%)
  adding: t5-mqg-base-model/added_tokens.json (deflated 83%)
  adding: t5-mqg-base-model/model.safetensors (deflated 8%)
  adding: t5-mqg-base-model/special_tokens_map.json (deflated 85%)
  adding: t5-mqg-base-model/config.json (deflated 63%)
  adding: t5-mqg-base-model/training_args.bin (deflated 51%)
  adding: t5-mqg-base-model/spiece.model (deflated 48%)
  adding: t5-mqg-base-model/generation_config.json (deflated 29%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
!zip -r t5-mqg-base.zip t5-mqg-base

  adding: t5-mqg-base/ (stored 0%)
  adding: t5-mqg-base/checkpoint-8364/ (stored 0%)
  adding: t5-mqg-base/checkpoint-8364/tokenizer_config.json (deflated 94%)
  adding: t5-mqg-base/checkpoint-8364/added_tokens.json (deflated 83%)
  adding: t5-mqg-base/checkpoint-8364/model.safetensors (deflated 8%)
  adding: t5-mqg-base/checkpoint-8364/rng_state.pth (deflated 25%)
  adding: t5-mqg-base/checkpoint-8364/special_tokens_map.json (deflated 85%)
  adding: t5-mqg-base/checkpoint-8364/config.json (deflated 63%)
  adding: t5-mqg-base/checkpoint-8364/training_args.bin (deflated 51%)
  adding: t5-mqg-base/checkpoint-8364/spiece.model (deflated 48%)
  adding: t5-mqg-base/checkpoint-8364/scaler.pt (deflated 60%)
  adding: t5-mqg-base/checkpoint-8364/scheduler.pt (deflated 55%)
  adding: t5-mqg-base/checkpoint-8364/optimizer.pt (deflated 9%)
  adding: t5-mqg-base/checkpoint-8364/trainer_state.json (deflated 70%)
  adding: t5-mqg-base/checkpoint-8364/generation_config.json (deflated 29%)
  adding: 